In [2]:
import pennylane as qml
import numpy as np

In [ ]:
n_qubits: int = 5
dev = qml.device('default.clifford', wires = n_qubits)

def RX(theta, wires):
    if theta == 0:
        qml.I(wires)
    elif theta == np.pi/2:
        qml.adjoint(qml.S(wires))
        qml.H(wires)
        qml.adjoint(qml.S(wires))
    elif theta == np.pi:
        qml.X(wires)
    elif theta ==  3*np.pi/2:
        qml.S(wires)
        qml.H(wires)
        qml.S(wires)
    elif theta ==  4*np.pi/2:
        RX(0, wires)
    elif theta ==  -1*np.pi/2:
        RX(3*np.pi/2, wires)
    else:
        raise ValueError(f"theta = {theta} makes e^itheta/2P not Clifford")

@qml.qnode(dev)
def circuit(thetas):
    for i in range(n_qubits):
        RX(thetas[i], wires = i)

    return [
        qml.expval(qml.Z(i)) for i in range(n_qubits)
    ]

In [73]:
possible_thetas = np.pi/2 * np.array([0,1,2,3])

np.random.choice(possible_thetas, 5)

array([1.57079633, 1.57079633, 4.71238898, 4.71238898, 3.14159265])

In [86]:
n_shots = 1000

def e(n, i):
    out = np.zeros(n)
    out[i] = 1
    return out

K_theta = []
for shot in range(n_shots):
    counter = 0
    thetas = np.random.choice(possible_thetas, n_qubits)
    for i in range(n_qubits):
        out1 = np.sum(circuit(thetas + np.pi/2 * e(n_qubits, i)))
        out2 = np.sum(circuit(thetas - np.pi/2 * e(n_qubits, i)))
        counter += (out1 - out2)**2
    K_theta.append(counter / 4)

In [87]:
np.mean(K_theta)

np.float64(2.537)

I should:
- Switch to integer mod 4 notation in my auxiliary RX
- Add non-|0> initial states
- Wrap the parameter-shift-rule-gradient into a function
- Memorize the $\hat{K}_\theta$ for each $theta$